In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import json
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [3]:
train_data_path = '/content/gdrive/MyDrive/dataset/training'
test_data_path = '/content/gdrive/MyDrive/dataset/test'

In [8]:
labels=[]
images=[]
total_img = 0
for file in os.listdir(train_data_path):
    img_path=os.path.join(train_data_path,file)
    current_label=file
    for img in os.listdir(img_path):
        if total_img%1000==0:
          print("Done {} images".format(total_img),file)
        image = plt.imread(os.path.join(img_path,img))
        labels.append(current_label)
        images.append(image)
        total_img+=1
print("Total Images: {}".format(total_img))

Done 0 images background
Done 1000 images background
Done 2000 images background
Done 3000 images background
Done 4000 images background
Done 5000 images hi
Total Images: 5819


In [9]:
IMAGE_SIZE = [64,64]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [10]:
x = Flatten()(vgg.output)
prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [11]:
model.compile(loss='binary_crossentropy',
              optimizer='adamax',
              metrics=['accuracy'])

In [17]:
en_labels = LabelEncoder().fit_transform(labels)

In [18]:
X = np.array(images)
y = np.array(to_categorical(en_labels))

In [20]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.1)

In [21]:
model_history=model.fit(
  X_train,y_train,
  validation_data=(X_val,y_val),
  epochs=50,
  steps_per_epoch=100)

Epoch 1/50
100/100 [==============================] - 54s 188ms/step - loss: 9.1712 - accuracy: 0.7552 - val_loss: 0.5444 - val_accuracy: 0.7663
Epoch 2/50
100/100 [==============================] - 15s 150ms/step - loss: 0.5460 - accuracy: 0.7652 - val_loss: 0.5138 - val_accuracy: 0.7663
Epoch 3/50
100/100 [==============================] - 15s 150ms/step - loss: 0.4836 - accuracy: 0.7777 - val_loss: 0.4224 - val_accuracy: 0.8196
Epoch 4/50
100/100 [==============================] - 15s 151ms/step - loss: 0.3736 - accuracy: 0.8452 - val_loss: 0.3330 - val_accuracy: 0.8780
Epoch 5/50
100/100 [==============================] - 15s 151ms/step - loss: 0.2223 - accuracy: 0.9129 - val_loss: 0.1424 - val_accuracy: 0.9450
Epoch 6/50
100/100 [==============================] - 15s 151ms/step - loss: 0.1255 - accuracy: 0.9544 - val_loss: 0.2278 - val_accuracy: 0.9433
Epoch 7/50
100/100 [==============================] - 15s 151ms/step - loss: 0.1215 - accuracy: 0.9601 - val_loss: 0.1297 - val_ac

In [22]:
test_imgs = []
file_name=[]
for file in os.listdir(test_data_path):
  img_path=os.path.join(test_data_path,file)
  image = plt.imread(img_path)
  test_imgs.append(image)
  file_name.append(file)

In [23]:
preds=model.predict(np.array(test_imgs))

In [24]:
A = []
for i in preds:
  print(i)
  A.append(np.argmax(i))

[1.0000000e+00 5.1913855e-20]
[1. 0.]
[1.0000000e+00 1.1428609e-20]
[1.000000e+00 5.839997e-34]
[1.0000000e+00 3.5350715e-30]
[1.0000000e+00 1.8965466e-28]
[1.0000000e+00 6.1625056e-26]
[1.000000e+00 2.728252e-32]
[1.0000000e+00 1.0445222e-31]
[1. 0.]
[1.0000000e+00 3.2293854e-28]
[1.0000000e+00 2.8246277e-21]
[1.0000000e+00 3.1375509e-18]
[1.000000e+00 1.736064e-30]
[1.0000000e+00 1.1581043e-25]
[1.0000000e+00 5.6886893e-22]
[1.0000000e+00 9.1532624e-32]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1.3286315e-37 1.0000000e+00]
[9.999999e-01 8.013158e-08]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[2.791694e-17 1.000000e+00]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[4.5761348e-07 9.9999952e-01]
[4.4134045e-29 1.0000000e+00]
[1.0412865e-11 1.0000000e+00]
[0. 1.]
[0. 1.]
[2.217367e-14 1.000000e+00]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1.578908e-23 1.000000e+00]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[2.0607956e-28 1.0000000e+00]
[0. 1.]
[0. 1.

In [28]:
ordered_file_name = []
ordered_labels = []
for i in range(1,99):
  for j in file_name:
    if i==eval(j[:-4]):
      ordered_file_name.append(j)
      ordered_labels.append(int(A[file_name.index(j)]))

In [29]:
final_result = dict(zip(ordered_file_name,ordered_labels))

In [30]:
with open("noobs_D_AI.json", "w") as outfile:
    json.dump(final_result, outfile)